In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import time

In [ ]:
# 사이트 로딩 함수
def call_site(page) :
    url = f'https://df.nexon.com/community/dnfboard/list?page={page}'
    headers = ({
        'USER-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'
    })
    time.sleep(1)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

# 게시글 크롤링
def call_content(title_num, page) :
    url = f'https://df.nexon.com/community/dnfboard/article/{title_num}?page={page}'
    headers = ({
        'USER-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'
    })
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [65]:
# 실행한 날짜는 시간 밖에 나오질 않아 연년월일을 추가 
now_time = dt.datetime.now()

# 공식 홈페이지를 몇 페이지까지 크롤링?
### 주제에 맞게끔 데이터를 분석할려면 24년 12월 ~ 베누스 업데이트 시간(2월) 까지는 수집이 필요함
#### 잘못하면 1000페이지 이상 해야할지도 모르니까 Time으로 잘 끊어서 해야할듯.

max_page =  3000 # 5000단위로 끊어서 진행

# 데이터 축적을 위한 list
category_lst = []
num_lst = []
nickname_lst = []
title_lst = []
time_lst = []
content_lst = []

# 크롤링 시작
for page in range(1, max_page) :

    # 데이터 로드
    soup = call_site(page)

    # 날짜 기준점보다 이전이면 크롤링 중단 플래그
    should_break = False

    # 각 정보 필터링 데이터 받을 때마다 진행
    for i in range(len(soup.select('article.board_list li.category'))) :

        # 각각의 필요한 정보 필터링
        category = soup.select('article.board_list li.category')[i].text # 글 카테고리
        title_num = soup.select('li[data-no]')[i]['data-no'] # 타이틀 넘버
        nickname = soup.select('article.board_list li.author')[i].text.strip()  # 유저 닉네임
        title = soup.select('article.board_list li.title')[i].text.strip().split()  # 글 제목
        title_time = soup.select('article.board_list li.date')[i].text # 게시글 작성시간

        if nickname == '던전앤파이터' :
            continue
        
        # title_time에 대한 전처리 필요(ex) 12:34 -> 2024.04.09)
        try :
            title_time = dt.datetime.strptime(title_time, "%M:%S")
            title_time = title_time.replace(year = now_time.year, month=now_time.month, day=now_time.day).strftime("%Y-%m-%d")
        except :
            title_time = dt.datetime.strptime(title_time, "%Y.%m.%d").strftime("%Y-%m-%d")

        # 날짜 필터링 - 2024년 12월 1일 이전 게시물은 크롤링 중단
        if title_time < '2024-12-01' :
            should_break = True
            break

        # 게시글 내용 가져오기
        content_soup = call_content(title_num, page)
        content = content_soup.find('div', class_ ='bd_viewcont').text.strip().split()

        # 데이터 축적
        category_lst.append(category)
        num_lst.append(title_num)
        nickname_lst.append(nickname)
        title_lst.append(title)
        time_lst.append(title_time)
        content_lst.append(content)

    if should_break :
        print("크롤링 끝")
        break

    home_df = pd.DataFrame({
        'time' : time_lst,
        'num' : num_lst,
        'category' : category_lst,
        'nickname' : nickname_lst,
        'title' : title_lst,
        'content' : content_lst
    })

print("크롤링 끝")

# 엑셀에서 읽어야 하니까 그냥 utf-8로 통일
home_df.to_csv('dnf_homepage_content.csv', encoding = 'utf-8')

크롤링 끝
크롤링 끝


# 크롤링 데이터 확인

In [12]:
df = pd.read_csv("dnf_homepage_content.csv", index_col='time').drop(columns='Unnamed: 0')
df.drop_duplicates(subset='content')

,num,category,nickname,title,content
time,,,,,
2025-04-09,2895383,수다,타쿠쟝,"['[공통]', '★★븜하', '커뮤니티', '오픈톡방', '인원', '모집', '...","['★★븜하', '커뮤니티', '오픈톡방', '인원', '모집', '[수다', '/..."
2025-04-09,2924227,질문,성기사주세요,"['[공통]', '던담', '딜', '관련질문', '(1)']","['새로고침', '누르면', '딜이', '낮아졌다', '높아졌다하는데', '왜이런건..."
2025-04-09,2924226,수다,이올라,"['[공통]', '갱신되는', '카드']","['나벨레이드', '갱신', '카드', '0']"
2025-04-09,2924225,질문,리홍뀨,"['[공통]', '무한화서', '내일', '끝나면', '에픽무기도', '못까는건가요...","['분홍색', '큰거', '하나만', '먹으면', '되는데내일', '끝나면', '없..."
2025-04-09,2924224,질문,Fiendish강유,"['[공통]', '패스', '및', '패키지', '관련', '질문', '(5)']","['안녕하세요복귀', '30일차', '뉴비입니다.현재', '프로스트', '패키지',..."
...,...,...,...,...,...
2024-12-01,2892482,질문,술끊은격투가,"['[여넨마스터]', '바린이', '질문염', '(2)']","['바칼무기제작상자인가를받아서', '너클을눌럿더니', '너클을3개를줫는데', '이걸..."
2024-12-01,2892481,질문,수라뇽뇽™,"['[공통]', '딜러1', '버퍼1', '새로하는', '뉴비입니다.', '(8)']","['이벤트템끼고', '골드노가다?', '를', '하는게', '맞을까요??', '한다..."
2024-12-01,2892480,질문,주황,"['[남크루세이더]', '안녕하세요', '이제', '뭘해야될지모르겠어서', '질문글...","['저는', '홀리랑,배틀메이지', '2캐릭을', '키우고', '있구요둘다', '총..."
